In [ ]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import sympy
from IPython.display import clear_output

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [ ]:
colors = plt.cm.magma(np.linspace(.3,1,8))

In [ ]:
def sparse_recovery(A,S,m):
    """
    sparse recovery algorithm from [ACHKMM24]
    """
    
    n,d = A.shape
    G = np.random.randn(d,m)
    Z = A@G

    A_ = np.zeros((n,d))

    for i in range(n):
        si = np.where(S[i])[0]
        A_[i,si] = np.linalg.lstsq(G[si].T,Z[i],rcond=None)[0]

    return A_

# Model Problem

In [ ]:
n = 1000
B = np.diag(4*np.ones(n)) + np.diag(-np.ones(n-1),1) + np.diag(-np.ones(n-1),-1)
A = np.linalg.inv(B)
A /= np.linalg.norm(A)

In [ ]:
bs = [0,1,2,3,4,5]
ms = np.geomspace(1,1e3+1,20,dtype=int)
n_trials = 20


err_all = {}
err_opt_all = {}
for b in bs:

    err = np.full((len(ms),n_trials),np.nan)

    s = 2*b+1
    S = np.eye(n,dtype=bool)
    for i in range(1,b+1):
        S += np.diag(np.ones(n-i,dtype=bool),i) + np.diag(np.ones(n-i,dtype=bool),-i)

    for i,m in enumerate(ms):
        print(f"{b},{m}")
        clear_output(wait=True)
        
        if m<s+2:
            continue
        for j in range(n_trials):
            A_ = sparse_recovery(A,S,m)
            err[i,j] = np.linalg.norm(A_ - A*S)

    err_all[b] = err
    err_opt_all[b] = np.linalg.norm(A-A*S)

In [ ]:
#np.save('model.npy',[err_all,err_opt_all],allow_pickle=True)
[err_all,err_opt_all] = np.load('model.npy',allow_pickle=True)

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(10,3.5))
plt.subplots_adjust(wspace=.3)

for i,b in enumerate(bs):
    c = colors[i]
    
    s = 2*b+1
    err = err_all[b]
    err_opt = err_opt_all[b]

    mean = np.mean(err**2,axis=1)**.5

    σ = 0.1
    minq = np.quantile(err,σ,axis=1)
    maxq = np.quantile(err,1-σ,axis=1)

    axs[0].plot(np.arange(np.max(ms)),err_opt+np.sqrt(s/(np.arange(np.max(ms))-s-1))*err_opt,color='k',ls=':')
    axs[0].plot(ms,err_opt+mean,marker='o',ms=4,color=c,ls='None',label=f'$s={s}$')
    axs[0].fill_between(ms,err_opt+minq,err_opt+maxq,alpha=.2,color=c);

    axs[0].set_yscale('log')
    axs[0].set_xscale('log')


    axs[1].plot(np.arange(np.max(ms)),np.sqrt(s/(np.arange(np.max(ms))-s-1))*err_opt,color='k',ls=':')
    axs[1].plot(ms,mean,marker='o',ms=4,color=c,ls='None',label=f'$s={s}$')
    axs[1].fill_between(ms,minq,maxq,alpha=.2,color=c);

    axs[1].set_yscale('log')
    axs[1].set_xscale('log')


axs[0].set_ylabel('approximation error: $\|\mathbf{A} - \widetilde{\mathbf{A}}\|_{\mathsf{F}}$')
axs[1].set_ylabel('sparse recovery error: $\|\mathbf{S}\circ \mathbf{A} - \widetilde{\mathbf{A}}\|_{\mathsf{F}}$')

axs[0].set_xlabel('number of matvecs $m$')
axs[1].set_xlabel('number of matvecs $m$')


plt.legend()

plt.savefig('imgs/inverse_decay.pdf')

# Trefethen primes

In [ ]:
n = 1000

b = 1

S = np.eye(n,dtype=bool)
for j in range(1,int(np.log2(n))):

    for i in range(2**j-b,2**j+b+1):
        S += np.diag(np.ones(n-i,dtype=bool),i) + np.diag(np.ones(n-i,dtype=bool),-i)

s = np.max(np.sum(S,axis=0))

In [ ]:
B = np.diag(list(sympy.primerange(2, sympy.prime(n) + 1)))
for j in range(1,int(np.log2(n))+1):
    i = 2**j
    B += np.diag(np.ones(n-i,dtype=bool),i) + np.diag(np.ones(n-i,dtype=bool),-i)


A = np.linalg.inv(B)

A /= np.linalg.norm(A)

In [ ]:
fig,axs = plt.subplots(2,1,figsize=(4,4),height_ratios=[1,36])
plt.subplots_adjust(hspace=.03)

axs[0].axis('off')

axs[1].imshow(np.abs(B),cmap='binary',norm=mpl.colors.LogNorm(vmin=1e-3,vmax=1e3))
axs[1].set_xticks([])
axs[1].set_yticks([])

plt.savefig('imgs/tref_mat_orig.pdf')

b=5
S = np.zeros((n,n),dtype=bool)
for j in range(0,int(np.log2(n))+1):
    for i in range(max(0,2**j-b),min(2**j+b,n)+1):
        S += np.diag(np.ones(n-i,dtype=bool),i) + np.diag(np.ones(n-i,dtype=bool),-i)


fig,axs = plt.subplots(2,1,figsize=(4,4),height_ratios=[1,36])
plt.subplots_adjust(hspace=.03)

axs[0].axis('off')

axs[1].imshow(np.abs(A),cmap='binary',norm=mpl.colors.LogNorm())
axs[1].set_xticks([])
axs[1].set_yticks([])

plt.savefig('imgs/tref_mat.pdf')

fig,axs = plt.subplots(2,1,figsize=(4,4),height_ratios=[1,36])
plt.subplots_adjust(hspace=.03)

axs[0].axis('off')

axs[1].imshow(S,cmap='binary')
axs[1].set_xticks([])
axs[1].set_yticks([])

plt.savefig('imgs/tref_sparsity.pdf')

In [ ]:
bs = [0,1,2,5,10]
ms = np.geomspace(1e1,5e2+1,20,dtype=int)
n_trials = 100


s_all = {}
err_all = {}
cond_all = {}
err_opt_all = {}
for b in bs:

    err = np.full((len(ms),n_trials),np.nan)
    cond = np.full((len(ms),n_trials),np.nan)

    S = np.zeros((n,n),dtype=bool)
    for j in range(0,int(np.log2(n))+1):
        for i in range(max(0,2**j-b),min(2**j+b,n)+1):
            S += np.diag(np.ones(n-i,dtype=bool),i) + np.diag(np.ones(n-i,dtype=bool),-i)

    s = np.max(np.sum(S,axis=0))
    s_all[b] = s
    
    for i,m in enumerate(ms):

        print(f"{b},{m}")
        clear_output(wait=True)

        if m<s+2: 
            continue
        for j in range(n_trials):
            A_ = sparse_recovery(A,S,m)
            err[i,j] = np.linalg.norm(A_ - A*S)

    err_all[b] = err
    err_opt_all[b] = np.linalg.norm(A-A*S)

In [ ]:
#np.save('trefethen_primes.npy',[err_all,err_opt_all,s_all],allow_pickle=True)
[err_all,err_opt_all,s_all] = np.load('trefethen_primes.npy',allow_pickle=True)

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(10,3.5))
plt.subplots_adjust(wspace=.3)

for i,b in enumerate(bs):
    c = colors[i]
    
    s = s_all[b]
    err = err_all[b]
    err_opt = err_opt_all[b]

    mean = np.mean(err**2,axis=1)**.5

    σ = 0.1
    minq = np.quantile(err,σ,axis=1)
    maxq = np.quantile(err,1-σ,axis=1)

    axs[0].plot(np.arange(np.max(ms)),err_opt+np.sqrt(s/(np.arange(np.max(ms))-s-1))*err_opt,color='k',ls=':');
    axs[0].plot(ms,err_opt+mean,marker='o',ms=4,color=c,ls='None',label=f'$s={s}$')
    axs[0].fill_between(ms,err_opt+minq,err_opt+maxq,alpha=.2,color=c);

    axs[0].set_yscale('log')
    axs[0].set_xscale('log')


    axs[1].plot(np.arange(np.max(ms)),np.sqrt(s/(np.arange(np.max(ms))-s-1))*err_opt,color='k',ls=':');
    axs[1].plot(ms,mean,marker='o',ms=4,color=c,ls='None',label=f'$s={s}$')
    axs[1].fill_between(ms,minq,maxq,alpha=.2,color=c);

    axs[1].set_yscale('log')
    axs[1].set_xscale('log')


axs[0].set_ylabel('approximation error: $\|\mathbf{A} - \widetilde{\mathbf{A}}\|_{\mathsf{F}}$')
axs[1].set_ylabel('sparse recovery error: $\|\mathbf{S}\circ \mathbf{A} - \widetilde{\mathbf{A}}\|_{\mathsf{F}}$')

axs[0].set_xlabel('number of matvecs $m$')
axs[1].set_xlabel('number of matvecs $m$')


plt.legend()

plt.savefig('imgs/tref_inv.pdf')